In [ ]:
import pandas as pd
from instagrapi import Client
from instagrapi.exceptions import LoginRequired, TwoFactorRequired, PleaseWaitFewMinutes
import getpass
import logging
import random
import time
import os

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def exponential_backoff(retry):
    """Calculate an exponential backoff time."""
    return 2 ** retry + random.uniform(0, 1)

def login_user():
    cl = Client()
    USERNAME, PASSWORD = get_instagram_credentials()
    try:
        cl.login(USERNAME, PASSWORD)
    except LoginRequired:
        logger.error("Login failed. Authentication challenge required.")
        return None
    except TwoFactorRequired:
        verification_code = input("Enter the two-factor authentication code: ")
        try:
            cl.login(USERNAME, PASSWORD, verification_code=verification_code)
        except TwoFactorRequired:
            logger.error("Invalid two-factor authentication code.")
            return None
    return cl

def get_instagram_credentials():
    username = input("Enter your Instagram username: ")
    password = getpass.getpass("Enter your Instagram password (input is hidden): ")
    return username, password

def fetch_instagram_followers(account_username):
    cl = login_user()
    if cl is None:
        return None

    user_id = cl.user_id_from_username(account_username)
    
    # Initialize variables for fetching followers
    all_followers = []
    max_id = ""
    retry = 0  # Initialize retry counter
    max_retries = 5  # Set maximum number of retries

    while retry < max_retries:
        try:
            followers_chunk, next_max_id = cl.user_followers_v1_chunk(user_id, max_id=max_id)
            all_followers.extend(followers_chunk)
            if not next_max_id:  # No more followers to fetch
                break
            max_id = next_max_id
            retry = 0  # Reset retry counter after a successful request
        except PleaseWaitFewMinutes:
            logger.warning(f"Rate limit hit. Waiting before retrying...")
            time.sleep(exponential_backoff(retry))
            retry += 1
            continue

    if retry == max_retries:
        logger.error("Maximum retries reached. Exiting...")
        return pd.DataFrame()  # Return empty DataFrame on failure

    follower_data = [{
        'username': follower.username,
        'full_name': follower.full_name,
        'user_id': follower.pk
    } for follower in all_followers]

    logger.info(f"Fetched {len(follower_data)} Instagram followers.")
    return pd.DataFrame(follower_data)

def save_to_csv(dataframe, filepath):
    try:
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        dataframe.to_csv(filepath, index=False)
        logger.info(f"Data saved to {filepath}")
    except OSError as e:
        logger.error(f"Cannot save file into {filepath}: {e}")

if __name__ == "__main__":
    account_username = "afadesign.official"
    followers_df = fetch_instagram_followers(account_username)
    if followers_df is not None and not followers_df.empty:
        save_to_csv(followers_df, "./followers.csv")
